In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_tx.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,810638,1063697200,4/15/20,671,PPP,PERCHERON HOLDINGS LLC,1904 W GRAND PKWY N,KATY,TX,77449.0,...,999 Other,5313 Activities Related to Real Estate,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,NaN
1,810639,1268757405,5/4/20,671,PPP,LINK STAFFING SERVICES CORPORATION,1800 BERING DR STE 800,HOUSTON,TX,77057.0,...,999 Other,5613 Employment Services,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,NaN
2,810640,1269717210,4/15/20,610,PPP,"GOODMAN NETWORKS, INC.",2801 NETWORK BLVD SUITE 300,FRISCO,TX,75034.0,...,999 Other,2382 Building Equipment Contractors,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,NaN
3,810641,1648657107,4/10/20,610,PPP,"ROSA'S CAF & TORTILLA FACTORY, LTD.",5000 Overton Plaza Suite 300,Fort Worth,TX,76109.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,NaN
4,810642,1657727204,4/15/20,610,PPP,"CENTRAL FREIGHT LINES, INC",5601 W WACO DR,WACO,TX,76710.0,...,999 Other,4841 General Freight Trucking,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76180,886818,9936567203,4/28/20,671,PPP,ST. ANTHONY CATHOLIC CHURCH,401 South Parker,Bryan,TX,77803.0,...,999 Other,8131 Religious Organizations,8131.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"401 South Parker, Bryan, TX",30.670456,-96.374926,NaN
76181,886819,9960748303,1/31/21,681,PPS,URCORE COMPANIES LLC,1386 Industrial Dr Ste 125,New Braunfels,TX,78130.0,...,999 Other,5614 Business Support Services,5614.0,SAN ANTONIO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,NaN
76182,886820,9966447007,4/9/20,678,PPP,STEADFAST ENERGY INVESTMENTS LP,4305 N. Garfield,Midland,TX,79705.0,...,999 Other,2111 Oil and Gas Extraction,2111.0,LUBBOCK DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,NaN
76183,886821,9967088603,3/26/21,677,PPP,RUDDOCK MANUFACTURING COMPANY INC.,1801 Magoffin Ave,El Paso,TX,79901.0,...,315 Apparel,3152 Cut and Sew Apparel Manufacturing,3152.0,EL PASO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
29844,6582497008,"806 PO Box, MELVIN, TX",NaN,NaN
40231,4283458309,"325, DALLAS, TX",NaN,NaN
70645,7955037106,"156 ENCANTADA, HORSESHOE LANE, TX",NaN,NaN
75408,4766927306,"2715 HANCOCK DR, AUSTIN, TX",NaN,NaN


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,1063697200,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741
1,1268757405,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116
2,1269717210,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479
3,1648657107,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639
4,1657727204,"5601 W WACO DR, WACO, TX",31.521110,-97.184595


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [3]:
fips_check_scraped = pd.read_csv("../../data/state_data/geo/geo_fips/TX_fips_scraped.csv") 
fips_check_scraped.head() #15869

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,810638,810638,810638,1063697200,4/15/20,671,PPP,PERCHERON HOLDINGS LLC,...,999 Other,5313 Activities Related to Real Estate,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14
1,1,1,810639,810639,810639,1268757405,5/4/20,671,PPP,LINK STAFFING SERVICES CORPORATION,...,999 Other,5613 Employment Services,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14
2,2,2,810640,810640,810640,1269717210,4/15/20,610,PPP,"GOODMAN NETWORKS, INC.",...,999 Other,2382 Building Equipment Contractors,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14
3,3,3,810641,810641,810641,1648657107,4/10/20,610,PPP,"ROSA'S CAF & TORTILLA FACTORY, LTD.",...,999 Other,7225 Restaurants and Other Eating Places,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14
4,4,4,810642,810642,810642,1657727204,4/15/20,610,PPP,"CENTRAL FREIGHT LINES, INC",...,999 Other,4841 General Freight Trucking,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,4.830900e+14


In [4]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [10]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [5]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,1063697200,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14
1,1268757405,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14
2,1269717210,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14
3,1648657107,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14
4,1657727204,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,4.830900e+14
...,...,...,...,...,...
76180,9936567203,"401 South Parker, Bryan, TX",30.670456,-96.374926,NaN
76181,9960748303,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,NaN
76182,9966447007,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,NaN
76183,9967088603,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,NaN


In [6]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

,LoanNumber,full_add,Lat,Long,FIPS_z
8871,7032338307,"1 Cowboys Way Ste 350, Frisco, TX",33.109812,-96.829992,NaN
12449,8415257008,"9225 King James Drive, DALLAS, TX",32.835728,-96.880204,NaN
16529,1206667309,"4275 Little Road Ste 106, Arlington, TX",32.679294,-97.194621,NaN
19603,3906658400,"4306 Yoakum Blvd Ste 600, Houston, TX",29.734040,-95.393060,NaN
19635,1875617302,"112 PECAN ST STE 1212, SAN ANTONIO, TX",29.428421,-98.492189,NaN
...,...,...,...,...,...
76180,9936567203,"401 South Parker, Bryan, TX",30.670456,-96.374926,NaN
76181,9960748303,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,NaN
76182,9966447007,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,NaN
76183,9967088603,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,NaN


<h4>FIPS Script

In [7]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [9]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
#driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))
#executable_path = {'executable_path': webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))}
executable_path = {'executable_path': ChromeDriverManager().install()}
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', **executable_path, options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('48')] #TX
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

[WDM] - Downloading: 100%|██████████| 8.84M/8.84M [00:00<00:00, 11.7MB/s]


  0%|          | 0/51128 [00:00<?, ?it/s]

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=111.0.5563.64)
Stacktrace:
0   chromedriver                        0x000000010f054428 chromedriver + 4899880
1   chromedriver                        0x000000010efd1a23 chromedriver + 4364835
2   chromedriver                        0x000000010ec1bbf6 chromedriver + 474102
3   chromedriver                        0x000000010ec033c4 chromedriver + 373700
4   chromedriver                        0x000000010ec03133 chromedriver + 373043
5   chromedriver                        0x000000010ec1d782 chromedriver + 481154
6   chromedriver                        0x000000010eca027e chromedriver + 1016446
7   chromedriver                        0x000000010ec85333 chromedriver + 906035
8   chromedriver                        0x000000010ec4f55f chromedriver + 685407
9   chromedriver                        0x000000010ec50a7e chromed

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [1]:
geo_id_list =geo_id 

NameError: name 'geo_id' is not defined

Length Check

In [20]:
print(len(geo_id_list))

25062


Matching FIPS to DataFrame

In [21]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:25062] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null

In [22]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
0,1063697200,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,[482015429014002]
1,1268757405,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,[482014314042001]
2,1269717210,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,[480850305041027]
3,1648657107,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,[484391054055006]
4,1657727204,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,[483090023022005]
...,...,...,...,...,...
25057,1207967209,"1904 Joe Stephens Drive, WESLACO, TX",26.178562,-97.969759,[482770008003008]
25058,3399558606,"6612 Avenue U, Houston, TX",29.751128,-95.303839,[482014301022007]
25059,4613537110,"6767 PORTWEST DR Suite 120, HOUSTON, TX",29.779985,-95.446373,[480291212034000]
25060,9899627001,"1598 COPPERFIELD PKWY, COLLEGE STATION, TX",30.646201,-96.290423,[484230007002017]


Remove brackets

In [23]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
0,1063697200,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,482015429014002
1,1268757405,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,482014314042001
2,1269717210,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,480850305041027
3,1648657107,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,484391054055006
4,1657727204,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,483090023022005
...,...,...,...,...,...
25057,1207967209,"1904 Joe Stephens Drive, WESLACO, TX",26.178562,-97.969759,482770008003008
25058,3399558606,"6612 Avenue U, Houston, TX",29.751128,-95.303839,482014301022007
25059,4613537110,"6767 PORTWEST DR Suite 120, HOUSTON, TX",29.779985,-95.446373,480291212034000
25060,9899627001,"1598 COPPERFIELD PKWY, COLLEGE STATION, TX",30.646201,-96.290423,484230007002017


In [24]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
8871,7032338307,"1 Cowboys Way Ste 350, Frisco, TX",33.109812,-96.829992,NaN
12449,8415257008,"9225 King James Drive, DALLAS, TX",32.835728,-96.880204,NaN
16529,1206667309,"4275 Little Road Ste 106, Arlington, TX",32.679294,-97.194621,NaN
19603,3906658400,"4306 Yoakum Blvd Ste 600, Houston, TX",29.734040,-95.393060,NaN
19635,1875617302,"112 PECAN ST STE 1212, SAN ANTONIO, TX",29.428421,-98.492189,NaN
20069,5021797010,"2300 E HASTINGS AVE, AMARILLO, TX",35.249128,-101.807919,NaN
20561,2212327106,"818 Prototype Rd, FLATONIA, TX",29.694050,-97.120481,NaN
23739,6748737107,"3505 5 S. BUCKNER BLVD, DALLAS, TX",32.713838,-96.683110,NaN
24999,2109577309,"2163 Golden Heights Rd Ste 405, FORT WORTH, TX",32.931912,-97.327332,NaN


Parse for Merge

In [25]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,1063697200,482015429014002
1,1268757405,482014314042001
2,1269717210,480850305041027
3,1648657107,484391054055006
4,1657727204,483090023022005
...,...,...
25057,1207967209,482770008003008
25058,3399558606,482014301022007
25059,4613537110,480291212034000
25060,9899627001,484230007002017


Read in base data

In [27]:
#if from geocode file
#fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_tx.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
#fips_check.head()

#if from FIPS file
fips_check = pd.read_csv("data/state_data/geo/geo_fips/TX_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head() #22826


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,810638,810638,810638,1063697200,4/15/20,671,PPP,PERCHERON HOLDINGS LLC,1904 W GRAND PKWY N,KATY,...,999 Other,5313 Activities Related to Real Estate,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,NaN
1,810639,810639,810639,1268757405,5/4/20,671,PPP,LINK STAFFING SERVICES CORPORATION,1800 BERING DR STE 800,HOUSTON,...,999 Other,5613 Employment Services,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,NaN
2,810640,810640,810640,1269717210,4/15/20,610,PPP,"GOODMAN NETWORKS, INC.",2801 NETWORK BLVD SUITE 300,FRISCO,...,999 Other,2382 Building Equipment Contractors,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,NaN
3,810641,810641,810641,1648657107,4/10/20,610,PPP,"ROSA'S CAF & TORTILLA FACTORY, LTD.",5000 Overton Plaza Suite 300,Fort Worth,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,NaN
4,810642,810642,810642,1657727204,4/15/20,610,PPP,"CENTRAL FREIGHT LINES, INC",5601 W WACO DR,WACO,...,999 Other,4841 General Freight Trucking,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,NaN


Merge with MainFrame

In [28]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,810638,810638,810638,1063697200,4/15/20,671,PPP,PERCHERON HOLDINGS LLC,1904 W GRAND PKWY N,KATY,...,5313 Activities Related to Real Estate,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,NaN,482015429014002
1,810639,810639,810639,1268757405,5/4/20,671,PPP,LINK STAFFING SERVICES CORPORATION,1800 BERING DR STE 800,HOUSTON,...,5613 Employment Services,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,NaN,482014314042001
2,810640,810640,810640,1269717210,4/15/20,610,PPP,"GOODMAN NETWORKS, INC.",2801 NETWORK BLVD SUITE 300,FRISCO,...,2382 Building Equipment Contractors,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,NaN,480850305041027
3,810641,810641,810641,1648657107,4/10/20,610,PPP,"ROSA'S CAF & TORTILLA FACTORY, LTD.",5000 Overton Plaza Suite 300,Fort Worth,...,7225 Restaurants and Other Eating Places,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,NaN,484391054055006
4,810642,810642,810642,1657727204,4/15/20,610,PPP,"CENTRAL FREIGHT LINES, INC",5601 W WACO DR,WACO,...,4841 General Freight Trucking,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,NaN,483090023022005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76180,886818,886818,886818,9936567203,4/28/20,671,PPP,ST. ANTHONY CATHOLIC CHURCH,401 South Parker,Bryan,...,8131 Religious Organizations,8131.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"401 South Parker, Bryan, TX",30.670456,-96.374926,NaN,NaN
76181,886819,886819,886819,9960748303,1/31/21,681,PPS,URCORE COMPANIES LLC,1386 Industrial Dr Ste 125,New Braunfels,...,5614 Business Support Services,5614.0,SAN ANTONIO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,NaN,NaN
76182,886820,886820,886820,9966447007,4/9/20,678,PPP,STEADFAST ENERGY INVESTMENTS LP,4305 N. Garfield,Midland,...,2111 Oil and Gas Extraction,2111.0,LUBBOCK DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,NaN,NaN
76183,886821,886821,886821,9967088603,3/26/21,677,PPP,RUDDOCK MANUFACTURING COMPANY INC.,1801 Magoffin Ave,El Paso,...,3152 Cut and Sew Apparel Manufacturing,3152.0,EL PASO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,NaN,NaN


<b>Fill FIPS Columns - if FIPS file

In [29]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,810638,810638,810638,1063697200,4/15/20,671,PPP,PERCHERON HOLDINGS LLC,1904 W GRAND PKWY N,KATY,...,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,NaN,482015429014002,482015429014002
1,810639,810639,810639,1268757405,5/4/20,671,PPP,LINK STAFFING SERVICES CORPORATION,1800 BERING DR STE 800,HOUSTON,...,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,NaN,482014314042001,482014314042001
2,810640,810640,810640,1269717210,4/15/20,610,PPP,"GOODMAN NETWORKS, INC.",2801 NETWORK BLVD SUITE 300,FRISCO,...,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,NaN,480850305041027,480850305041027
3,810641,810641,810641,1648657107,4/10/20,610,PPP,"ROSA'S CAF & TORTILLA FACTORY, LTD.",5000 Overton Plaza Suite 300,Fort Worth,...,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,NaN,484391054055006,484391054055006
4,810642,810642,810642,1657727204,4/15/20,610,PPP,"CENTRAL FREIGHT LINES, INC",5601 W WACO DR,WACO,...,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,NaN,483090023022005,483090023022005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76180,886818,886818,886818,9936567203,4/28/20,671,PPP,ST. ANTHONY CATHOLIC CHURCH,401 South Parker,Bryan,...,8131.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"401 South Parker, Bryan, TX",30.670456,-96.374926,NaN,NaN,NaN
76181,886819,886819,886819,9960748303,1/31/21,681,PPS,URCORE COMPANIES LLC,1386 Industrial Dr Ste 125,New Braunfels,...,5614.0,SAN ANTONIO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,NaN,NaN,NaN
76182,886820,886820,886820,9966447007,4/9/20,678,PPP,STEADFAST ENERGY INVESTMENTS LP,4305 N. Garfield,Midland,...,2111.0,LUBBOCK DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,NaN,NaN,NaN
76183,886821,886821,886821,9967088603,3/26/21,677,PPP,RUDDOCK MANUFACTURING COMPANY INC.,1801 Magoffin Ave,El Paso,...,3152.0,EL PASO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,NaN,NaN,NaN


Drop Extra FIPS Columns

In [30]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,810638,810638,810638,1063697200,4/15/20,671,PPP,PERCHERON HOLDINGS LLC,1904 W GRAND PKWY N,KATY,...,999 Other,5313 Activities Related to Real Estate,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,482015429014002
1,810639,810639,810639,1268757405,5/4/20,671,PPP,LINK STAFFING SERVICES CORPORATION,1800 BERING DR STE 800,HOUSTON,...,999 Other,5613 Employment Services,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,482014314042001
2,810640,810640,810640,1269717210,4/15/20,610,PPP,"GOODMAN NETWORKS, INC.",2801 NETWORK BLVD SUITE 300,FRISCO,...,999 Other,2382 Building Equipment Contractors,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,480850305041027
3,810641,810641,810641,1648657107,4/10/20,610,PPP,"ROSA'S CAF & TORTILLA FACTORY, LTD.",5000 Overton Plaza Suite 300,Fort Worth,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,484391054055006
4,810642,810642,810642,1657727204,4/15/20,610,PPP,"CENTRAL FREIGHT LINES, INC",5601 W WACO DR,WACO,...,999 Other,4841 General Freight Trucking,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,483090023022005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76180,886818,886818,886818,9936567203,4/28/20,671,PPP,ST. ANTHONY CATHOLIC CHURCH,401 South Parker,Bryan,...,999 Other,8131 Religious Organizations,8131.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"401 South Parker, Bryan, TX",30.670456,-96.374926,NaN
76181,886819,886819,886819,9960748303,1/31/21,681,PPS,URCORE COMPANIES LLC,1386 Industrial Dr Ste 125,New Braunfels,...,999 Other,5614 Business Support Services,5614.0,SAN ANTONIO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,NaN
76182,886820,886820,886820,9966447007,4/9/20,678,PPP,STEADFAST ENERGY INVESTMENTS LP,4305 N. Garfield,Midland,...,999 Other,2111 Oil and Gas Extraction,2111.0,LUBBOCK DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,NaN
76183,886821,886821,886821,9967088603,3/26/21,677,PPP,RUDDOCK MANUFACTURING COMPANY INC.,1801 Magoffin Ave,El Paso,...,315 Apparel,3152 Cut and Sew Apparel Manufacturing,3152.0,EL PASO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,NaN


In [31]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
8871,819509,819509,819509,7032338307,1/27/21,610,PPS,ESRP ADVISORY DALLAS LLC,1 Cowboys Way Ste 350,Frisco,...,999 Other,5312 Offices of Real Estate Agents and Brokers,5312.0,DALLAS / FT WORTH DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1 Cowboys Way Ste 350, Frisco, TX",33.109812,-96.829992,NaN
12449,823087,823087,823087,8415257008,4/8/20,610,PPP,"TRINITY HARDWOOD DISTRIBUTORS, INC",9225 King James Drive,DALLAS,...,999 Other,4232 Furniture and Home Furnishing Merchant Wh...,4232.0,DALLAS / FT WORTH DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"9225 King James Drive, DALLAS, TX",32.835728,-96.880204,NaN
16529,827167,827167,827167,1206667309,4/28/20,610,PPP,"ALL CARE PROFESSIONAL HOME HEALTH, INC.",4275 Little Road Ste 106,Arlington,...,999 Other,6216 Home Health Care Services,6216.0,DALLAS / FT WORTH DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"4275 Little Road Ste 106, Arlington, TX",32.679294,-97.194621,NaN
19603,830241,830241,830241,3906658400,2/5/21,671,PPS,VISTA ENERGY MARKETING LP,4306 Yoakum Blvd Ste 600,Houston,...,999 Other,2212 Natural Gas Distribution,2212.0,HOUSTON DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"4306 Yoakum Blvd Ste 600, Houston, TX",29.734040,-95.393060,NaN
19635,830273,830273,830273,1875617302,4/28/20,681,PPP,MULE RESTAURANT GROUP LLC,112 PECAN ST STE 1212,SAN ANTONIO,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SAN ANTONIO DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"112 PECAN ST STE 1212, SAN ANTONIO, TX",29.428421,-98.492189,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76180,886818,886818,886818,9936567203,4/28/20,671,PPP,ST. ANTHONY CATHOLIC CHURCH,401 South Parker,Bryan,...,999 Other,8131 Religious Organizations,8131.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"401 South Parker, Bryan, TX",30.670456,-96.374926,NaN
76181,886819,886819,886819,9960748303,1/31/21,681,PPS,URCORE COMPANIES LLC,1386 Industrial Dr Ste 125,New Braunfels,...,999 Other,5614 Business Support Services,5614.0,SAN ANTONIO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,NaN
76182,886820,886820,886820,9966447007,4/9/20,678,PPP,STEADFAST ENERGY INVESTMENTS LP,4305 N. Garfield,Midland,...,999 Other,2111 Oil and Gas Extraction,2111.0,LUBBOCK DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,NaN
76183,886821,886821,886821,9967088603,3/26/21,677,PPP,RUDDOCK MANUFACTURING COMPANY INC.,1801 Magoffin Ave,El Paso,...,315 Apparel,3152 Cut and Sew Apparel Manufacturing,3152.0,EL PASO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,NaN


Convert to File

In [32]:
fips_TX = fips_merge_drop
#fips_TN = fips_merge_drop

In [33]:
fips_TX.to_csv("../../data/state_data/geo/geo_fips/TX_fips_scraped.csv") 

Review & Compare

In [36]:
geo_TX_fips = pd.read_csv("../../data/state_data/geo/geo_fips/TX_fips_scraped.csv") 
geo_TX_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,810638,810638,810638,1063697200,4/15/20,671,PPP,PERCHERON HOLDINGS LLC,1904 W GRAND PKWY N,...,999 Other,5313 Activities Related to Real Estate,5313.0,HOUSTON DISTRICT OFFICE,e $5-10 million,500 or more,"1904 W GRAND PKWY N, KATY, TX",29.802433,-95.773741,4.820154e+14
1,1,810639,810639,810639,1268757405,5/4/20,671,PPP,LINK STAFFING SERVICES CORPORATION,1800 BERING DR STE 800,...,999 Other,5613 Employment Services,5613.0,HOUSTON DISTRICT OFFICE,e $5-10 million,100 to 249,"1800 BERING DR STE 800, HOUSTON, TX",29.748200,-95.480116,4.820143e+14
2,2,810640,810640,810640,1269717210,4/15/20,610,PPP,"GOODMAN NETWORKS, INC.",2801 NETWORK BLVD SUITE 300,...,999 Other,2382 Building Equipment Contractors,2382.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"2801 NETWORK BLVD SUITE 300, FRISCO, TX",33.102777,-96.827479,4.808503e+14
3,3,810641,810641,810641,1648657107,4/10/20,610,PPP,"ROSA'S CAF & TORTILLA FACTORY, LTD.",5000 Overton Plaza Suite 300,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5000 Overton Plaza Suite 300, Fort Worth, TX",32.687155,-97.397639,4.843911e+14
4,4,810642,810642,810642,1657727204,4/15/20,610,PPP,"CENTRAL FREIGHT LINES, INC",5601 W WACO DR,...,999 Other,4841 General Freight Trucking,4841.0,DALLAS / FT WORTH DISTRICT OFFICE,e $5-10 million,500 or more,"5601 W WACO DR, WACO, TX",31.521110,-97.184595,4.830900e+14


In [37]:
##### fill in missing Lat, Long, FIPS
geo_TX_fips.loc[geo_TX_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
8871,8871,819509,819509,819509,7032338307,1/27/21,610,PPS,ESRP ADVISORY DALLAS LLC,1 Cowboys Way Ste 350,...,999 Other,5312 Offices of Real Estate Agents and Brokers,5312.0,DALLAS / FT WORTH DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1 Cowboys Way Ste 350, Frisco, TX",33.109812,-96.829992,NaN
12449,12449,823087,823087,823087,8415257008,4/8/20,610,PPP,"TRINITY HARDWOOD DISTRIBUTORS, INC",9225 King James Drive,...,999 Other,4232 Furniture and Home Furnishing Merchant Wh...,4232.0,DALLAS / FT WORTH DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"9225 King James Drive, DALLAS, TX",32.835728,-96.880204,NaN
16529,16529,827167,827167,827167,1206667309,4/28/20,610,PPP,"ALL CARE PROFESSIONAL HOME HEALTH, INC.",4275 Little Road Ste 106,...,999 Other,6216 Home Health Care Services,6216.0,DALLAS / FT WORTH DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"4275 Little Road Ste 106, Arlington, TX",32.679294,-97.194621,NaN
19603,19603,830241,830241,830241,3906658400,2/5/21,671,PPS,VISTA ENERGY MARKETING LP,4306 Yoakum Blvd Ste 600,...,999 Other,2212 Natural Gas Distribution,2212.0,HOUSTON DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"4306 Yoakum Blvd Ste 600, Houston, TX",29.734040,-95.393060,NaN
19635,19635,830273,830273,830273,1875617302,4/28/20,681,PPP,MULE RESTAURANT GROUP LLC,112 PECAN ST STE 1212,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SAN ANTONIO DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"112 PECAN ST STE 1212, SAN ANTONIO, TX",29.428421,-98.492189,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76180,76180,886818,886818,886818,9936567203,4/28/20,671,PPP,ST. ANTHONY CATHOLIC CHURCH,401 South Parker,...,999 Other,8131 Religious Organizations,8131.0,HOUSTON DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"401 South Parker, Bryan, TX",30.670456,-96.374926,NaN
76181,76181,886819,886819,886819,9960748303,1/31/21,681,PPS,URCORE COMPANIES LLC,1386 Industrial Dr Ste 125,...,999 Other,5614 Business Support Services,5614.0,SAN ANTONIO DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1386 Industrial Dr Ste 125, New Braunfels, TX",29.716146,-98.096696,NaN
76182,76182,886820,886820,886820,9966447007,4/9/20,678,PPP,STEADFAST ENERGY INVESTMENTS LP,4305 N. Garfield,...,999 Other,2111 Oil and Gas Extraction,2111.0,LUBBOCK DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"4305 N. Garfield, Midland, TX",32.032663,-102.112495,NaN
76183,76183,886821,886821,886821,9967088603,3/26/21,677,PPP,RUDDOCK MANUFACTURING COMPANY INC.,1801 Magoffin Ave,...,315 Apparel,3152 Cut and Sew Apparel Manufacturing,3152.0,EL PASO DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1801 Magoffin Ave, El Paso, TX",31.767618,-106.468716,NaN


In [39]:
geo_TX_fips["BusinessType"] = geo_TX_fips["BusinessType"].fillna('Not Available')
geo_TX_fips["BusinessType"].unique()
geo_TX_fips.to_csv("../../data/state_data/geo/geo_fips/TX_fips_scraped.csv") 
geo_TX_fips =pd.read_csv("../../data/state_data/geo/geo_fips/TX_fips_scraped.csv") 
geo_TX_fips["BusinessType"].unique()

array(['Limited Liability Company(LLC)', 'Subchapter S Corporation',
       'Corporation', 'Partnership', 'Limited Liability Partnership',
       '501(c)3 – Non Profit', 'Employee Stock Ownership Plan(ESOP)',
       'Professional Association', 'Sole Proprietorship',
       'Non-Profit Organization', 'Not Available', 'Joint Venture',
       'Cooperative', '501(c)6 – Non Profit Membership',
       'Self-Employed Individuals', 'Trust', 'Independent Contractors',
       'Non-Profit Childcare Center', 'Single Member LLC',
       'Tenant in Common', 'Qualified Joint-Venture (spouses)'],
      dtype=object)